In [27]:
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import pickle

import tensorflow as tf
from keras.applications.resnet_v2 import *
from keras.applications.vgg19 import *
from keras.preprocessing.image_dataset import *

In [17]:
train_i = []
test_i = []

train_xy = pd.read_csv('train.csv')
train_path = train_xy['id'].values
test_path = pd.read_csv('imagenames.csv')['id'].values

# for f in train_path:
#     i = cv2.imread('./train/' + f + '.jpg')
#     train_i.append(i)
# for f in test_path:
#     i = cv2.imread('./test/' + f + '.jpg')
#     test_i.append(i)

In [19]:
#model = ResNet152V2(weights = 'imagenet', include_top = False)
model = VGG19(weights='imagenet', include_top=False)

In [28]:
train_preds = ...#[]
test_preds = ...#[]

# for i in tqdm(train_i):
#     inp = preprocess_input(np.expand_dims(i, axis=0))
#     pred = model.predict(inp)
#     train_preds.append(pred)

# for i in tqdm(test_i):
#     inp = preprocess_input(np.expand_dims(i, axis=0))
#     pred = model.predict(inp)
#     test_preds.append(pred)

100%|██████████| 1200/1200 [51:38<00:00,  2.58s/it]


In [31]:
# f = open(f'./test_vgg19.pckl','wb')
# pickle.dump(test_preds,f)
# f.close()

# f = open(f'./train_vgg19.pckl','wb')
# pickle.dump(train_preds,f)
# f.close()

In [65]:
#np.append(np.array([[4,5,6]]),[[1,2,3]],axis=0)
train_xy.iloc[np.argsort([1,2,3])[-3:]]['id'].values

array(['IMG2744_1', 'IMG2744_2', 'IMG2744_3'], dtype=object)

In [67]:
best_train = []
for test in tqdm(test_preds):
    match = [np.sum((train-test)**2)**0.5 for train in train_preds]
    best_match = np.argsort(match)[-3:]
    row = train_xy.iloc[best_match]['id'].values
    best_train.append(row)

df = pd.DataFrame(best_train, columns=['3','2','1'])
df.to_csv('vgg19_matches.csv',index=False)

100%|██████████| 1200/1200 [57:38<00:00,  2.88s/it]
